In [2]:
import sys
sys.path.append('../src')
from transformer_encoder import TransformerEncoderV3
import seaborn as sns
import numpy as np
from datasets import load_dataset
from transformers import BertTokenizer
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Input, Dense, Dropout
from tensorflow.keras.models import Model

2024-05-10 12:11:59.156994: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-10 12:11:59.181559: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-10 12:11:59.874866: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-10 12:12:03.074979: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-10 12:12:09.799237: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [7]:
from datasets import load_dataset

# Load an example dataset, 'wikipedia' for English, 2020-03-01 version
dataset = load_dataset("wikipedia", "20220301.en", split=["train"])
# print(dataset[0])
article_texts_dataset = dataset[0]['text'][:1000]

# Extract text and write to a file
with open('input_text.txt', 'w', encoding='utf-8') as f:
    for article  in article_texts_dataset:
        # Write each Wikipedia article on a new line
        f.write(article.replace('\n', ' ') + '\n')


In [8]:
# print(dataset[0]['text'][1])

In [9]:
!python create_pretraining_data.py --vocab_file vocab.txt --input_text input_text.txt --output_tfrecord output.tfrecord --do_lower_case --nsp


/mnt/d/mydev/attention/wslvenv2nd/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1988: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(
Processed 9990 instances and saved to output.tfrecord


In [13]:
import tensorflow as tf

def _parse_function(proto):
    # Define your tfrecord again. It must be the same as the one used for saving your data.
    feature_description = {
        'input_ids': tf.io.FixedLenFeature([128], tf.int64),  # Assuming input_ids are of length 128
        'segment_ids': tf.io.FixedLenFeature([128], tf.int64),  # Assuming segment_ids are of length 128
        'masked_lm_positions': tf.io.VarLenFeature(tf.int64),
        'masked_lm_labels': tf.io.VarLenFeature(tf.int64),
        'next_sentence_labels': tf.io.FixedLenFeature([], tf.int64),
    }

    # Load one example
    parsed_features = tf.io.parse_single_example(proto, feature_description)
    
    # Turn your sparse array into a dense array with default values as 0
    parsed_features['masked_lm_positions'] = tf.sparse.to_dense(parsed_features['masked_lm_positions'], default_value=0)
    parsed_features['masked_lm_labels'] = tf.sparse.to_dense(parsed_features['masked_lm_labels'], default_value=0)

    return parsed_features
# Read the TFRecord file
def load_dataset(file_path):
    dataset = tf.data.TFRecordDataset(file_path)
    dataset = dataset.map(_parse_function)  # Parse the record into tensors.
    return dataset

# Path to the TFRecord file
tfrecord_file_path = 'output.tfrecord'

# Load the dataset
parsed_dataset = load_dataset(tfrecord_file_path)
# Display a few examples from the dataset
for parsed_record in parsed_dataset.take(50):  # Only take first 5 examples
    print('Input IDs:', parsed_record['input_ids'].numpy())
    print('Segment IDs:', parsed_record['segment_ids'].numpy())
    print('Masked LM Positions:', parsed_record['masked_lm_positions'].numpy())
    print('Masked LM Labels:', parsed_record['masked_lm_labels'].numpy())
    print('Next Sentence Label:', parsed_record['next_sentence_labels'].numpy())
    print('---')


Input IDs: [  101  9617 11140  2964  2003   103  2576  4695   103  2929  2008  2003
  8040 23606  7476  1997  3691  1998   103  2035 26097  1010 24873 11890
  3512  3596 24576 12571  1012  9617 11140  2964  4455  2005  1996 15766
  1997  1996  2110  1010  7971  2009  4324  2000  2022 14203  1010   103
  2229  7895  3468   103  1998 17631  1012  2004  1037  7145  2187  1011
  3358   103  1010   103   102 19465  2003  1037 11265 10976 24844 18349
 24073  2389  8761   103  2011  8190  2007   103  8290  1998  4807  1010
  1998  2011   103 12219 23563   103  1012  3008  2411   103   103  2076
  1996  2034  2093  2086  1997  2037  2775  1005  1055  2166  1012  2122
  5751  2411 26164  6360  1010  2295  2070  8740 16774  2594  2336  3325
 26237  1999  2037   103  1998  2591  4813   102]
Segment IDs: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

2024-05-10 12:43:42.897758: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
